<h2 style="text-align: center">Final exam Image Processing </h2>
<h3 style="text-align: center">Sarawin Buakaew ID: 63-010725-2017-2</h3>




In [2]:
import numpy as np
import cv2 as cv
from cv2 import aruco
import os 
import matplotlib.pyplot as plt

In [3]:
def preprocessing(img) :
    img_gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    # img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img_gray

In [4]:
# params_dir = os.getcwd()+'camera_params/monocular_camera_params'
# print(params_dir)
# # Loaa camera parameters
# ret = np.load(params_dir+'ret.npy')


In [5]:
vid = cv.VideoCapture('Final_exam/Dataset-1/left_output-1.avi')
template_img = cv.imread('Templates/Template-1.png')
template_gray = preprocessing(template_img)

# plt.imshow(template_gray)

In [6]:
sift = cv.SIFT_create()
bf = cv.BFMatcher()

In [7]:
def feature_object_detection(template_img,query_gray, frame, min_match_number) :
    template_kpts, template_desc = sift.detectAndCompute(template_img, None)
    query_kpts, query_desc = sift.detectAndCompute(query_gray, None)
    matches = bf.knnMatch(template_desc, query_desc, k=2)
    
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.7*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])
    
    if len(good_matches) > min_match_number :
        src_pts = np.float32([ template_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv.findHomography(src_pts, dst_pts, cv.RANSAC, 10.0) # H RANSAC
        # get the bounding box around template image
        h, w = template_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2)
        transformed_box = cv.perspectiveTransform(template_box, H)

        detected_img = cv.polylines(frame, [np.int32(transformed_box)], True, (0,0,255), 3, cv.LINE_AA)
        drawmatch_img = cv.drawMatchesKnn(template_img, template_kpts, detected_img, query_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

        return detected_img, drawmatch_img
    else :
        print('Keypoints not enough')
        return


<h4 style="text-align: center">VidoCapture is OpenVideo</h4>

In [8]:
while vid.isOpened():
    
    ret, frame = vid.read()
    
    if ret :
        query_img = frame
        kernel = np.array([[0,0,0],
                           [0,1.5,0],
                           [0,0,0]])
        filter = cv.filter2D(query_img, -1,kernel)
        query_gray = preprocessing(filter)
        detected, drawmatch =  feature_object_detection(template_img, query_gray, frame, 10)
    
        cv.imshow('Video exam',drawmatch)
        # if cv.waitKey(int(1000/20)) & 0xFF == ord('q') : # int(1000/20) ทำให้ video slowly
        if cv.waitKey(33) & 0xFF == ord('q'):
            break
    else :
            break

vid.release()
cv.destroyAllWindows()

Keypoints not enough


TypeError: cannot unpack non-iterable NoneType object

: 